# 任务说明
服饰图像描述,训练一个模型,对输入的服饰图片,输出描述信息，我们实现的模型有以下三个实现：
- ARCTIC，一个典型的基于注意力的编解码模型
- 视觉Transformer (ViT) + Transformer解码器
- 网格/区域表示、Transformer编码器+Transformer解码器
  
同时也实现三种测评方法进行测评：
- BLEU (Bilingual Evaluation Understudy)
- SPICE (Semantic Propositional Image Caption Evaluation): 
- CIDEr-D (Consensus-based Image Description Evaluation)

# 实验数据
数据集使用的是 DeepFashion-MultiModal (https://github.com/yumingj/DeepFashion-MultiModal), 仅用到image和textual descriptions ，数据集划分为40k+行数据的训练集和2k+行数据的测试集，`train_captions.json`和`test_captions.json`分别对应训练集和测试集的图片与描述信息的键值对应

In [8]:
img_path = f'../data/deepfashion-multimodal/images'
def cap_to_wvec(vocab,cap):#将文本描述转换成向量
    cap.replace(",","")
    cap.replace(".","")
    cap=cap.split()
    res=[]
    for word in cap:
        if word in vocab.keys():
            res.append(vocab[word])
        else: #不在字典的词
            res.append(vocab['<unk>'])
    return res
def wvec_to_cap(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    res=" ".join(res)
    return res
def wvec_to_capls(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    return res
class ImageTextDataset(Dataset):
    def __init__(self, dataset_path, vocab_path, split, captions_per_image=1, max_len=93, transform=None):

        self.split = split
        assert self.split in {'train', 'test'}
        self.cpi = captions_per_image
        self.max_len = max_len

        # 载入数据集
        with open(dataset_path, 'r') as f:
            self.data = json.load(f) #key是图片名字 value是描述
            self.data_img=list(self.data.keys())
        # 载入词典
        with open(vocab_path, 'r') as f:
            self.vocab = json.load(f)

        # PyTorch图像预处理流程
        self.transform = transform

        # Total number of datapoints
        self.dataset_size = len(self.data_img)

    def __getitem__(self, i):
        # 第i个文本描述对应第(i // captions_per_image)张图片
        img = Image.open(img_path+"/"+self.data_img[i]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        c_vec=cap_to_wvec(self.vocab,self.data[self.data_img[i]])
        #加入起始和结束标志
        c_vec = [self.vocab['<start>']] + c_vec + [self.vocab['<end>']]
        caplen = len(c_vec)
        caption = torch.LongTensor(c_vec+ [self.vocab['<pad>']] * (self.max_len + 2 - caplen))
        
        return img, caption, caplen
        
    def __len__(self):
        return self.dataset_size
def mktrainval(data_dir, vocab_path, batch_size, workers=1):
    train_tx = transforms.Compose([
        transforms.Resize(256), # 重置图像分辨率
        transforms.RandomCrop(224), # 随机裁剪
        transforms.ToTensor(), # 转换成Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化--三个参数为三个通道的均值和标准差
    ])
    val_tx = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    train_set = ImageTextDataset(os.path.join(data_dir, 'train_captions.json'), vocab_path, 'train',  transform=train_tx)
    test_set = ImageTextDataset(os.path.join(data_dir, 'test_captions.json'), vocab_path, 'test', transform=val_tx)

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    
    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True, drop_last=False)

    return train_loader, test_loader    
train_loader,test_loader=mktrainval(data_dir='../data/deepfashion-multimodal',\
                                        vocab_path='../data/deepfashion-multimodal/vocab.json',\
                                        batch_size=3,workers=0) 
#workers=0 是因为ipynb不支持多线程

# 实验环境
- Python  3.9.16
- 主要依赖库
  - torch 
  - torchvision
  - nltk 
实际使用py库情况如下

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence # 压紧填充序列
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet101_Weights
from nltk.translate.bleu_score import corpus_bleu # BLEU评价指标
import numpy as np
import json
from torch.utils.data import Dataset
import os
from PIL import Image
from collections import Counter,defaultdict
from argparse import Namespace 


# 所用的方法或模型

## 评估方法

### BLEU (BiLingual Evaluation Understudy)
- BLUE是比较常用的评估指标之一，也是我们默认指标，需要注意的是，再调用计算BLEU值之前，要先将文本中人工添加的文本开始符、结束符和占位符去掉，其公式如下， 实际代码中我们借助nltk库进行实现
$$BLEU = \sum_{n=1}^k w_n \frac{ngram_{sys}(n)}{ngram_{ref}(n)}$$
其中：
  - n 是 n-gram 的阶数，取值范围为 1 到 4。
  - wn 是 n-gram 的权重，通常取均匀权重。
  - ngramsys(n) 是机器翻译结果中的 n-gram 数量。
  - ngramref(n) 是参考翻译中的 n-gram 数量。
  BLEU 的得分范围为 0 到 1。得分越高，表示机器翻译结果与参考翻译越相似。
  - 优点：容易计算
  - 缺点:
    - 没有考虑n-gram的顺序
    - 平等对待所有的n-gram
    - 衡量的是句子之间的流畅性而非语义相似度
### CIDEr-D (Consensus-based Image Description Evaluation)
- 是CIDEr的改进，对于动词原形和名词匹配成功的问题，CIDEr-D不再取词根
其用了一种折扣函数来降低长句子对评分的影响，增加了惩罚生成句子和参考句子的长度差别的权重，并且通过对n-gram计数的截断操作不再计算生成句子中出现次数超过参考句子的n-gram,
从而减少了重复单词对评分的影响，其实也是计算1到4 gram的结果的平均值，其公式如下
$$C I D E r - D _ { n } ( c _ { i } , S _ { i } ) = \frac { 1 0 } { m } \sum _ { j } e ^ { - \frac { -( i ( c _ { i } ) - l ( s _ { i j } ) ) ^ { 2 } } { 2 \sigma ^ { 2 } } } \times \frac { \min ( g ^ { n } ( c _ { i } ) , g ^ { n } ( s _ { i j } ) ) \cdot g ^ { n } ( s _ { i j } ) } {| | g ^ { n } ( c _ { i } ) | | | g ^ { n } ( s _ { i j } ) || } $$
- 优点：
  - CIDEr引入了TF-IDF为n-gram进行加权，这样就避免评价候选句子时因为一些常见却不够有信息量的n-gram打上高分
- 缺点：
  - CIDEr取词根的操作会让一些动词的原型和名词匹配成功
  - 高置信度的词重复出现的长句的CIDEr得分也很高
### SPICE (Semantic Propositional Image Caption Evaluation): 
- 是以名词为中心的度量，是以图的语义表示来编码图像描述中的对象、属性和关系
首先要将候选句子和参考句子集转化为场景图
然后比较候选句子和参考句子集中元组的precision、recall，最终计算出F1 score
公式如下
$$SPICE = \sum_{i=1}^m \frac{1}{|S_i|} \sum_{j=1}^n \frac{s_{ij}}{|R_i|}
$$
  - m 是图像描述的数量。
  - n 是图像描述中的对象、属性和关系的数量。
  - Si 是图像描述 i 中的对象、属性和关系。
  - Ri 是参考图像描述 i 中的对象、属性和关系。
  - sij 是图像描述 i 中的对象、属性和关系 j 与参考图像描述 i 中的对象、属性和关系 j 的相似度
- 优点：
  - 在语义而非n-gram层级度量
  - 每个句子映射到场景图后可以从中提取出模型关于某些关系或者属性的识别能力
- 缺点
  - 缺少n-gram来度量句子的流畅性
  - 度量的准确性受到场景图解析器的制约

使用代码如下，在evaluate的时候调用,接受cands, refs返回对应评估分数

In [12]:
def cider_d(reference_list, candidate_list, n=4):
    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_cider_d(reference_list, candidate_list, n):
        cider_d_scores = []
        for refs, cand in zip(reference_list, candidate_list):
            cider_d_score = 0.0
            for i in range(1, n + 1):
                cand_ngrams = count_ngrams(cand, i)
                ref_ngrams_list = [count_ngrams(ref, i) for ref in refs]

                total_ref_ngrams = [ngram for ref_ngrams in ref_ngrams_list for ngram in ref_ngrams]

                count_cand = 0
                count_clip = 0

                for ngram in cand_ngrams:
                    count_cand += 1
                    if ngram in total_ref_ngrams:
                        count_clip += 1

                precision = count_clip / count_cand if count_cand > 0 else 0.0
                recall = count_clip / len(total_ref_ngrams) if len(total_ref_ngrams) > 0 else 0.0

                beta = 1.0
                f_score = (1 + beta**2) * precision * recall / (beta**2 * precision + recall) if precision + recall > 0 else 0.0

                cider_d_score += f_score

            cider_d_score /= n
            cider_d_scores.append(cider_d_score)

        return cider_d_scores

    reference_tokens_list = reference_list
    candidate_tokens_list = candidate_list

    scores = compute_cider_d(reference_tokens_list, candidate_tokens_list, n)

    return np.mean(scores)
def spice(reference_list, candidate_list, idf=None, beta=3):
    def tokenize(sentence):
        return sentence.lower().split()

    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_spice_score(reference, candidate, idf, beta):
        reference_tokens = reference
        candidate_tokens = candidate

        reference_ngrams = [count_ngrams(reference_tokens, i) for i in range(1, beta + 1)]
        candidate_ngrams = [count_ngrams(candidate_tokens, i) for i in range(1, beta + 1)]

        precision_scores = []
        recall_scores = []

        for i in range(beta):
            common_ngrams = set(candidate_ngrams[i]) & set(reference_ngrams[i])

            precision = len(common_ngrams) / len(candidate_ngrams[i]) if len(candidate_ngrams[i]) > 0 else 0.0
            recall = len(common_ngrams) / len(reference_ngrams[i]) if len(reference_ngrams[i]) > 0 else 0.0

            precision_scores.append(precision)
            recall_scores.append(recall)

        precision_avg = np.mean(precision_scores)
        recall_avg = np.mean(recall_scores)

        spice_score = (precision_avg * recall_avg) / (precision_avg + recall_avg) if precision_avg + recall_avg > 0 else 0.0

        if idf:
            spice_score *= np.exp(np.sum([idf[token] for token in common_ngrams]) / len(candidate_tokens))

        return spice_score

    if idf is None:
        idf = {}

    spice_scores = []

    for reference, candidate in zip(reference_list, candidate_list):
        spice_score = compute_spice_score(reference, candidate, idf, beta)
        spice_scores.append(spice_score)

    return np.mean(spice_scores)
def get_BLEU_score(cands, refs): #获取BLEU分数
    multiple_refs = []
    for idx in range(len(refs)):
        multiple_refs.append(refs[(idx//1)*1 : (idx//1)*1+1])#每个候选文本对应cpi==1条参考文本
    bleu4 = corpus_bleu(multiple_refs, cands, weights=(0.25,0.25,0.25,0.25))
    return bleu4
def get_CIDER_D_score(cands, refs): #获得CIDER-D分数
    refs_ = [wvec_to_capls(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_capls(model.vocab,cand) for cand in cands]
    return cider_d(refs_, cands_)
def get_SPICE_score(cands, refs): #获得SPICE分数
    refs_ = [wvec_to_cap(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_cap(model.vocab,cand) for cand in cands]
    return spice(refs_, cands_)

## 模型定义
### ARCTIC，一个典型的基于注意力的编解码模型

模型架构: ARCTIC 是一个基于注意力的编码-解码模型，使用了图像编码器和注意力解码器。图像编码器使用了 ResNet-101 网络进行特征提取。
注意力机制: 模型使用加性注意力机制，它接受查询（Q）和键值对（K，V），计算注意力分数，最后输出上下文向量。
解码器: 解码器采用 GRU，利用注意力上下文向量和当前时刻的词嵌入来生成预测结果。该模型支持束搜索来生成更准确的描述。


In [13]:
ARCTIC_config = Namespace(
        max_len = 93,
        captions_per_image = 1,
        batch_size = 32,
        image_code_dim = 2048,
        word_dim = 512,
        hidden_size = 512,
        attention_dim = 512,
        num_layers = 1,
        encoder_learning_rate = 0.0001,
        decoder_learning_rate = 0.0005,
        num_epochs = 10,
        grad_clip = 5.0,
        alpha_weight = 1.0,
        evaluate_step = 900, # 每隔多少步在验证集上测试一次
        checkpoint = None, # 如果不为None，则利用该变量路径的模型继续训练
        best_checkpoint = 'model/ARCTIC/best_ARCTIC.ckpt', # 验证集上表现最优的模型的路径
        last_checkpoint = 'model/ARCTIC/last_ARCTIC.ckpt', # 训练完成时的模型的路径
        beam_k = 5 #束搜索的束宽
    )
class ImageEncoder(nn.Module):
    def __init__(self, finetuned=True):
        super(ImageEncoder, self).__init__()
        model = torchvision.models.resnet101(weights=ResNet101_Weights.DEFAULT)
        # ResNet-101网格表示提取器
        self.grid_rep_extractor = nn.Sequential(*(list(model.children())[:-2])) #去掉最后两层 
        for param in self.grid_rep_extractor.parameters(): #冻结参数--不参与训练
            param.requires_grad = finetuned #是否微调
    def forward(self, images):
        out = self.grid_rep_extractor(images) 
        return out
class AdditiveAttention(nn.Module): #加性注意力
    def  __init__(self, query_dim, key_dim, attn_dim):
        """
            query_dim: 查询Q的维度
            key_dim: 键K的维度
            attn_dim: 注意力函数隐藏层表示的维度
        """
        
        super(AdditiveAttention, self).__init__()
        self.attn_w_1_q = nn.Linear(query_dim, attn_dim) #Q的线性变换
        self.attn_w_1_k = nn.Linear(key_dim, attn_dim) #K的线性变换
        self.attn_w_2 = nn.Linear(attn_dim, 1) #注意力函数隐藏层到输出层的线性变换
        self.tanh = nn.Tanh() #激活函数
        self.softmax = nn.Softmax(dim=1) #归一化函数

    def forward(self, query, key_value):
        """
        Q K V：Q和K算出相关性得分，作为V的权重，K=V
        参数：
            query: 查询 (batch_size, q_dim)
            key_value: 键和值，(batch_size, n_kv, kv_dim)
        """
        # （2）计算query和key的相关性，实现注意力评分函数
        # -> (batch_size, 1, attn_dim)
        queries = self.attn_w_1_q(query).unsqueeze(1) 
        # -> (batch_size, n_kv, attn_dim)
        keys = self.attn_w_1_k(key_value) #
        # -> (batch_size, n_kv)
        attn = self.attn_w_2(self.tanh(queries+keys)).squeeze(2)  #注意力评分函数
        # （3）归一化相关性分数
        # -> (batch_size, n_kv)
        attn = self.softmax(attn)  #归一化
        # （4）计算输出
        # (batch_size x 1 x n_kv)(batch_size x n_kv x kv_dim)
        # -> (batch_size, 1, kv_dim)
        output = torch.bmm(attn.unsqueeze(1), key_value).squeeze(1)
        return output, attn
class AttentionDecoder(nn.Module):
    def __init__(self, image_code_dim, vocab_size, word_dim, attention_dim, hidden_size, num_layers, dropout=0.5):
        super(AttentionDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, word_dim) #词嵌入 
        self.attention = AdditiveAttention(hidden_size, image_code_dim, attention_dim) #注意力机制
        self.init_state = nn.Linear(image_code_dim, num_layers*hidden_size) #初始化隐状态
        self.rnn = nn.GRU(word_dim + image_code_dim, hidden_size, num_layers) #GRU
        self.dropout = nn.Dropout(p=dropout) #dropout
        self.fc = nn.Linear(hidden_size, vocab_size) #全连接层
        # RNN默认已初始化
        self.init_weights() #初始化权重
        
    def init_weights(self): #初始化权重
        self.embed.weight.data.uniform_(-0.1, 0.1) #词嵌入
        self.fc.bias.data.fill_(0) #全连接层
        self.fc.weight.data.uniform_(-0.1, 0.1) #全连接层
    
    def init_hidden_state(self, image_code, captions, cap_lens):
        """
        参数：
            image_code：图像编码器输出的图像表示 
                        (batch_size, image_code_dim, grid_height, grid_width)
        """
        # 将图像网格表示转换为序列表示形式 
        batch_size, image_code_dim = image_code.size(0), image_code.size(1)
        # -> (batch_size, grid_height, grid_width, image_code_dim) 
        image_code = image_code.permute(0, 2, 3, 1)  
        # -> (batch_size, grid_height * grid_width, image_code_dim)
        image_code = image_code.view(batch_size, -1, image_code_dim)
        # （1）按照caption的长短排序
        sorted_cap_lens, sorted_cap_indices = torch.sort(cap_lens, 0, True)
        captions = captions[sorted_cap_indices]
        image_code = image_code[sorted_cap_indices]
         #（2）初始化隐状态
        hidden_state = self.init_state(image_code.mean(axis=1))
        hidden_state = hidden_state.view(
                            batch_size, 
                            self.rnn.num_layers, 
                            self.rnn.hidden_size).permute(1, 0, 2)
        return image_code, captions, sorted_cap_lens, sorted_cap_indices, hidden_state

    def forward_step(self, image_code, curr_cap_embed, hidden_state):
        #（3.2）利用注意力机制获得上下文向量
        # query：hidden_state[-1]，即最后一个隐藏层输出 (batch_size, hidden_size)
        # context: (batch_size, hidden_size)
        context, alpha = self.attention(hidden_state[-1], image_code)
        #（3.3）以上下文向量和当前时刻词表示为输入，获得GRU输出
        x = torch.cat((context, curr_cap_embed), dim=-1).unsqueeze(0)
        # x: (1, real_batch_size, hidden_size+word_dim)
        # out: (1, real_batch_size, hidden_size)
        out, hidden_state = self.rnn(x, hidden_state)
        #（3.4）获取该时刻的预测结果
        # (real_batch_size, vocab_size)
        preds = self.fc(self.dropout(out.squeeze(0)))
        return preds, alpha, hidden_state
        
    def forward(self, image_code, captions, cap_lens):
        """
        参数：
            hidden_state: (num_layers, batch_size, hidden_size)
            image_code:  (batch_size, feature_channel, feature_size)
            captions: (batch_size, )
        """
        # （1）将图文数据按照文本的实际长度从长到短排序
        # （2）获得GRU的初始隐状态
        image_code, captions, sorted_cap_lens, sorted_cap_indices, hidden_state \
            = self.init_hidden_state(image_code, captions, cap_lens)
        batch_size = image_code.size(0)
        # 输入序列长度减1，因为最后一个时刻不需要预测下一个词
        lengths = sorted_cap_lens.cpu().numpy() - 1
        # 初始化变量：模型的预测结果和注意力分数
        predictions = torch.zeros(batch_size, lengths[0], self.fc.out_features).to(captions.device)
        alphas = torch.zeros(batch_size, lengths[0], image_code.shape[1]).to(captions.device)
        # 获取文本嵌入表示 cap_embeds: (batch_size, num_steps, word_dim)
        cap_embeds = self.embed(captions)
        # Teacher-Forcing模式
        for step in range(lengths[0]):
            #（3）解码
            #（3.1）模拟pack_padded_sequence函数的原理，获取该时刻的非<pad>输入
            real_batch_size = np.where(lengths>step)[0].shape[0]
            preds, alpha, hidden_state = self.forward_step(
                            image_code[:real_batch_size], 
                            cap_embeds[:real_batch_size, step, :],
                            hidden_state[:, :real_batch_size, :].contiguous())            
            # 记录结果
            predictions[:real_batch_size, step, :] = preds
            alphas[:real_batch_size, step, :] = alpha
        return predictions, alphas, captions, lengths, sorted_cap_indices
    
class ARCTIC(nn.Module): #模型主体部分
    def __init__(self, image_code_dim, vocab, word_dim, attention_dim, hidden_size, num_layers):
        super(ARCTIC, self).__init__()
        self.vocab = vocab
        self.encoder = ImageEncoder()
        self.decoder = AttentionDecoder(image_code_dim, len(vocab),
                                        word_dim, attention_dim, hidden_size, num_layers)
        print("test")
    def forward(self, images, captions, cap_lens):
        image_code = self.encoder(images)
        return self.decoder(image_code, captions, cap_lens)
    def generate_by_beamsearch(self, images, beam_k, max_len): # beam_k束搜索
        vocab_size = len(self.vocab)
        image_codes = self.encoder(images)
        texts = []
        device = images.device
        # 对每个图像样本执行束搜索
        for image_code in image_codes:
            # 将图像表示复制k份
            image_code = image_code.unsqueeze(0).repeat(beam_k,1,1,1)
            # 生成k个候选句子，初始时，仅包含开始符号<start>
            cur_sents = torch.full((beam_k, 1), self.vocab['<start>'], dtype=torch.long).to(device)
            cur_sent_embed = self.decoder.embed(cur_sents)[:,0,:]
            sent_lens = torch.LongTensor([1]*beam_k).to(device)
            # 获得GRU的初始隐状态
            image_code, cur_sent_embed, _, _, hidden_state = \
                self.decoder.init_hidden_state(image_code, cur_sent_embed, sent_lens)
            # 存储已生成完整的句子（以句子结束符<end>结尾的句子）
            end_sents = []
            # 存储已生成完整的句子的概率
            end_probs = []
            # 存储未完整生成的句子的概率
            probs = torch.zeros(beam_k, 1).to(device)
            k = beam_k
            while True:
                preds, _, hidden_state = self.decoder.forward_step(image_code[:k], cur_sent_embed, hidden_state.contiguous())
                # -> (k, vocab_size)
                preds = nn.functional.log_softmax(preds, dim=1)
                # 对每个候选句子采样概率值最大的前k个单词生成k个新的候选句子，并计算概率
                # -> (k, vocab_size)
                probs = probs.repeat(1,preds.size(1)) + preds
                if cur_sents.size(1) == 1:
                    # 第一步时，所有句子都只包含开始标识符，因此，仅利用其中一个句子计算topk
                    values, indices = probs[0].topk(k, 0, True, True)
                else:
                    # probs: (k, vocab_size) 是二维张量
                    # topk函数直接应用于二维张量会按照指定维度取最大值，这里需要在全局取最大值
                    # 因此，将probs转换为一维张量，再使用topk函数获取最大的k个值
                    values, indices = probs.view(-1).topk(k, 0, True, True)
                # 计算最大的k个值对应的句子索引和词索引
                sent_indices = torch.div(indices, vocab_size, rounding_mode='trunc') 
                word_indices = indices % vocab_size 
                # 将词拼接在前一轮的句子后，获得此轮的句子
                cur_sents = torch.cat([cur_sents[sent_indices], word_indices.unsqueeze(1)], dim=1)
                # 查找此轮生成句子结束符<end>的句子
                end_indices = [idx for idx, word in enumerate(word_indices) if word == self.vocab['<end>']]
                if len(end_indices) > 0:
                    end_probs.extend(values[end_indices])
                    end_sents.extend(cur_sents[end_indices].tolist())
                    # 如果所有的句子都包含结束符，则停止生成
                    k -= len(end_indices)
                    if k == 0:
                        break
                # 查找还需要继续生成词的句子
                cur_indices = [idx for idx, word in enumerate(word_indices) 
                               if word != self.vocab['<end>']]
                if len(cur_indices) > 0:
                    cur_sent_indices = sent_indices[cur_indices]
                    cur_word_indices = word_indices[cur_indices]
                    # 仅保留还需要继续生成的句子、句子概率、隐状态、词嵌入
                    cur_sents = cur_sents[cur_indices]
                    probs = values[cur_indices].view(-1,1)
                    hidden_state = hidden_state[:,cur_sent_indices,:]
                    cur_sent_embed = self.decoder.embed(
                        cur_word_indices.view(-1,1))[:,0,:]
                # 句子太长，停止生成
                if cur_sents.size(1) >= max_len:
                    break
            if len(end_sents) == 0:
                # 如果没有包含结束符的句子，则选取第一个句子作为生成句子
                gen_sent = cur_sents[0].tolist()
            else: 
                # 否则选取包含结束符的句子中概率最大的句子
                gen_sent = end_sents[end_probs.index(max(end_probs))]
            texts.append(gen_sent)
        return texts
mode_arctic="../best_arctic.ckpt"
model=torch.load(mode_arctic)["model"] #加载模型

### 视觉Transformer (ViT) + Transformer解码器

### 网格/区域表示、Transformer编码器+Transformer解码器

# 实验结果
由于除了BLEU-4以外，其他指标的最大值其实和数据有关，所以为此我们先进行一个实验：得到如下指标的最大结果，然后将其作为一种相对的标准，结果如下：
BLEU-MAX:1.0|CIDEr-D-MAX:0.0043918896512309125|SPICE-MAX:0.18703616844830037，在评估过程中将会同时输出实际值和相对值（被压缩到0-1之间），

In [38]:
max_cider=0.0043918896512309125
max_spice=0.18703616844830037
def evaluate_(data_loader) :#用来测试剩下两个指标的最大值
    cands = []# 存储参考文本
    refs = []# 需要过滤的词
    filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']})
    for i, (imgs, caps, caplens) in enumerate(data_loader):
        cands.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()])# 参考文本
        refs.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()]) #候选文本
    bleu4_score = get_BLEU_score(cands, refs)
    cider_d_score = get_CIDER_D_score(cands, refs)
    spice_score= get_SPICE_score(cands, refs)
    print(f"BLEU-MAX:{bleu4_score}|CIDEr-D-MAX:{cider_d_score}|SPICE-MAX:{spice_score}")
    max_cider=cider_d_score
    max_spice=spice_score
evaluate_(test_loader)

BLEU-MAX:1.0|CIDEr-D-MAX:0.0043918896512309125|SPICE-MAX:0.18703616844830037


## ARCTIC的结果

In [45]:
def filter_useless_words(sent, filterd_words):
    # 去除句子中不参与BLEU值计算的符号
    return [w for w in sent if w not in filterd_words]
cider_d_score=0
spice_score=0
def evaluate(data_loader, model, config):
    model.eval()
    # 存储候选文本
    cands = []
    # 存储参考文本
    refs = []
    # 需要过滤的词
    filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']})
    cpi = config.captions_per_image
    device = next(model.parameters()).device
    for i, (imgs, caps, caplens) in enumerate(data_loader):
        with torch.no_grad():
            # 通过束搜索，生成候选文本
            texts = model.generate_by_beamsearch(imgs.to(device), config.beam_k, config.max_len+2)
            # 候选文本
            cands.extend([filter_useless_words(text, filterd_words) for text in texts])
            # 参考文本
            refs.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()])
    
    
    bleu4_score = get_BLEU_score(cands, refs)
    cider_d_score = get_CIDER_D_score(cands, refs)
    spice_score= get_SPICE_score(cands, refs)
    print(f"@@@实际值 BLEU:{bleu4_score}|CIDEr-D:{cider_d_score}|SPICE:{spice_score}")
    print(f"@@@相对值（0-1） BLEU:{bleu4_score}|CIDEr-D:{cider_d_score/max_cider}|SPICE:{spice_score/max_spice}")
    #model.train()
evaluate(test_loader, model, ARCTIC_config)

@@@实际值 BLEU:0.30466660274770024|CIDEr-D:0.0017447527516659645|SPICE:0.1336081641272871
@@@相对值（0-1） BLEU:0.30466660274770024|CIDEr-D:0.3972669830575009|SPICE:0.7143439968629298


同时我们以测试集的第一行数据来进行演示：以生成的描述文本和参考文本进行比较，直观评估ARCTIC的性能

In [24]:
def batch_eva(data_loader, model, config): #这里使用实验数据的第一个batch来进行演示
    model.eval()
    for i, (imgs, caps, caplens) in  enumerate(test_loader):
        cands = [] # 存储候选文本 
        refs = [] # 存储参考文本
        filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']}) #过滤词
        cpi = config.captions_per_image
        texts = model.generate_by_beamsearch(imgs.to("cuda"), config.beam_k, config.max_len+2)
        print("@生成数据：",texts)
        print("@生成的文本：",wvec_to_cap(model.vocab,texts[0])) #抽出一个batch的第一个文本
        print("@实际的文本：",wvec_to_cap(model.vocab,caps[0].tolist()))
        break
batch_eva(test_loader, model, ARCTIC_config)

@生成数据： [[154, 21, 47, 16, 31, 32, 39, 39, 52, 28, 10, 11, 12, 1, 39, 52, 16, 28, 7, 8, 9, 72, 13, 16, 84, 1, 49, 50, 28, 7, 8, 9, 10, 11, 12, 57, 16, 25, 59, 34, 35, 60, 155], [154, 21, 47, 16, 31, 32, 39, 39, 52, 28, 43, 12, 1, 39, 52, 16, 28, 7, 8, 9, 72, 13, 16, 84, 1, 49, 50, 28, 7, 8, 9, 43, 12, 57, 16, 25, 59, 34, 35, 60, 155], [154, 38, 40, 4, 5, 6, 7, 8, 9, 43, 12, 1, 13, 14, 15, 16, 81, 1, 18, 3, 16, 14, 5, 19, 1, 8, 16, 7, 9, 15, 4, 43, 12, 1, 26, 3, 16, 28, 7, 8, 9, 43, 12, 1, 26, 3, 16, 28, 7, 8, 9, 43, 12, 57, 16, 25, 59, 34, 35, 60, 155]]
@生成的文本： This person is wearing a tank tank top with solid color patterns. The tank top is with cotton fabric and its neckline is round. The pants are with cotton fabric and solid color patterns. There is an accessory on her wrist.
@实际的文本： This woman is wearing a tank tank shirt with graphic patterns and a three-point shorts. The tank shirt is with cotton fabric and its neckline is crew. The shorts are with cotton fabric and graphic patte

# 实验结果分析

根据实验结果，分析模型的优缺点
- BLEU-4 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 BLEU-4 上表现良好，得分为（0.30466660274770024）。
    模型生成的文本与参考文本的匹配度较高，语法和词汇的准确性得到了有效提升。
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
      ViT + Transformer 模型在 BLEU-4 上的得分为（具体的BLEU-4分数）。模型在服饰描述任务中取得了良好的结果，生成文本在语和词汇方面表现出色。
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:  
      网格/区域表示 + Transformer 模型在 BLEU-4 上的得分为（具体的BLEU-4分数）。
      模型在服饰描述任务中呈现出良好的BLEU-4性能，对服饰的描述与参考文本相符。
- CIDEr-D 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 CIDEr-D 上取得了（0.0017447527516659645） 相对值为 0.3972669830575009。
    CIDEr-D 分数显示模型在文本多样性和丰富性方面有一定的成功。
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
    ViT + Transformer 模型在 CIDEr-D 上的得分为（具体的CIDEr-D得分）。
    模型在服饰描述任务中具有较高的文本多样性和丰富性，体现了对不同服饰场景的准确描述。
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:
    网格/区域表示 + Transformer 模型在 CIDEr-D 上的得分为（具体的CIDEr-D得分）。
    模型对服饰描述的多样性和丰富性取得了一定的成功。
- SPICE 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 SPICE 上取得了（0.1336081641272871） 相对值为 0.7143439968629298。
    SPICE 分数反映了模型生成文本与图像内容相关性的程度，高分表明模型成功捕捉了图像的语义信息。
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
    ViT + Transformer 模型在 SPICE 上的得分为（具体的SPICE得分）。
    模型在服饰描述任务中表现出色，成功捕捉图像语义信息。
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:
    网格/区域表示 + Transformer 模型在 SPICE 上的得分为（具体的SPICE得分）。
    模型在 SPICE 上的表现显示其在描述图像内容方面的良好性能。

当然 ARCTIC、视觉Transformer (ViT) + Transformer解码器和网格/区域表示、Transformer编码器+Transformer解码器这三个模型有不同的优劣势

- ARCTIC 模型：
  - 优势：
    结合了注意力机制和编解码模型，有助于捕捉输入图像和生成描述之间的语义关系。
    注意力机制使得模型在生成描述时能够更加关注与服饰相关的区域，提高了描述的准确性。
  - 劣势：
    可能需要更多的计算资源和训练时间，因为结合了多个模型组件。
    在处理大规模数据集时，训练和推理速度可能较慢。
    推理过程中尝试了不同的生成方式，发现加入 beam search 策略效果最佳。


- 视觉 Transformer (ViT) + Transformer 解码器：
  - 优势：
    ViT 模型将输入图像转换为序列数据，直接应用 Transformer 解码器进行描述生成。
    Transformer 解码器在自然语言处理任务中表现出色，生成准确且流畅的描述。
  - 劣势：
    ViT 模型可能对输入图像的分辨率和细节要求较高，对于复杂的服饰图像可能需要更多的训练数据和计算资源。
    在处理长序列数据时，Transformer 解码器可能面临较长的训练和推理时间。


- 网格/区域表示、Transformer 编码器+Transformer 解码器：
  - 优势：
    网格/区域表示将图像划分为网格或区域，有助于捕捉局部特征。
    Transformer 编码器和解码器在处理序列数据时具有较强的建模能力，能够生成准确的描述。
  - 劣势：
    网格/区域表示可能需要额外的预处理步骤来划分图像，并可能导致信息损失。
    Transformer 编码器和解码器的训练和推理时间可能较长，特别是在处理大规模数据集时

## 存在的问题 

# 总结

在服饰描述任务上，三个模型各自具有独特的优势和劣势，选择适合任务需求的模型应全面考虑以下几个方面：

- 任务需求： 根据任务的具体需求，是否需要模型具有较强的语义关系捕捉能力或者对图像的全局信息有更好的处理能力。
- 数据集规模： 数据集规模的大小会影响模型的训练效果，因此需要根据具体数据集的情况进行选择
- 计算资源： 不同模型对计算资源的需求不同，需要根据实际可用的硬件条件进行选择,作为学生，我们的算力水平非常有限
- 推理速度： 在测试的过程中，推理速度其实区别比较明显，如果任务对推理速度有较高要求，需要选择相对较快的模型。

- 模型表现:
  - 模型在整体上表现良好，取得了令人满意的图像描述生成结果。
  - BLEU-4、CIDEr-D 和 SPICE 的综合分析揭示了模型在不同方面的表现。
- 实验收获:
  - 从实验中我们学到了模型在特定场景下的优势，并发现了一些未被预期的模型行为。
  - 实验结果与初步假设的一致性和不一致性提供了对模型表现的更深入认识。我们也对这一实际训练模型有了更加深入的了解
- 改进方向: 
  - 可以通过调整超参数或模型架构来提高对特殊场景的适应性。
  - 进一步的训练技巧或引入更多上下文信息可能有助于提高生成文本的连贯性。
- 未来工作:
  - 实验结果启示了未来进一步研究的方向，可以考虑拓展模型应用领域或在其他相关任务上进行深入研究。
  - 通过对实验结果的深入分析，未来的工作可以更有针对性地改进模型，以提高其在复杂场景下的性能。

